In [98]:
import gurobipy as gb
from gurobipy import*
import openpyxl  
import numpy as np
import math
import time
import copy
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import heapq


In [99]:
# PARA
M = 10E3
ISSme = 1

In [100]:
def get_values(filename,id = 0,delme = 0):
    wb = openpyxl.load_workbook(filename)
    sheet_names = wb.sheetnames
    print(sheet_names)

    sheet = wb[sheet_names[id]]
    maxrow = sheet.max_row 
    maxcol = sheet.max_column 
    value = sheet.cell(row=2, column=1).value
    alldata = []

    for row in sheet.iter_rows(min_row=1, max_row=maxrow, min_col=1, max_col=maxcol): # type: ignore
        row_data = [cell.value for cell in row]
        alldata.append(row_data)
    station_name = alldata[0][1:]

    if delme:
        del alldata[0]
        for i in range(len(alldata)):
            del alldata[i][0]


    return alldata,station_name

In [101]:
def refineme(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] is None:
                data[i][j] = M
    return data

In [102]:
def convme(data,title):
    degree = {}
    alldegree = 0
    for i,row in enumerate(data):
        
        degree[title[i]] = sum(row)
        alldegree = alldegree + sum(row)
    return degree,alldegree


In [103]:
def findme_maxdegreee(degree):
    max_degree = max(degree.values())

    max_degree_node = []
    for key in degree:
        if int(degree[key]) == int(max_degree):
            max_degree_node.append(key)
    return(max_degree,max_degree_node)

In [104]:
def picme(station_name,MyOD):
    plt.figure(figsize=(10, 6))
    MyOD = np.array(MyOD)
    plt.imshow(MyOD, cmap='Blues', aspect='auto', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xticks(ticks=np.arange(len(station_name)), labels=station_name, rotation=45)
    plt.yticks(ticks=np.arange(MyOD.shape[0]), labels=[f'Row {i+1}' for i in range(MyOD.shape[0])])
    plt.title('Heatmap of Matrix')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.tight_layout()
    plt.show()

In [105]:
MyOD,station_name = get_values('smallOD.xlsx',delme=1)

['Sheet1']


In [106]:
MyOD = refineme(MyOD)

In [107]:
station_name.insert(0,'d')
station_name.append('a')

In [108]:
station_name

['d', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'a']

In [109]:
# 设置插入的值
value_to_insert = 100

# 插入新列（在每行的开头和结尾都插入100）
for row in MyOD:
    row.insert(0, value_to_insert)  # 插入开头的列
    row.append(value_to_insert)     # 插入结尾的列

# 插入新行（在开头和结尾插入100的行）
new_row = [value_to_insert] * len(MyOD[0])  # 创建一个新行，列数为MyOD每行的列数
MyOD.insert(0, new_row)  # 在开头插入新行
MyOD.append(new_row)     # 在结尾插入新行

# 打印结果
for row in MyOD:
    print(row)


[100, 100, 100, 100, 100, 100, 100, 100]
[100, 10000.0, 10000.0, 10000.0, 1, 1, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 1, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 1, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 100, 100, 100, 100, 100, 100, 100]


In [110]:
for row in MyOD:
    print(row)

[100, 100, 100, 100, 100, 100, 100, 100]
[100, 10000.0, 10000.0, 10000.0, 1, 1, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 1, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 10000.0, 1, 10000.0, 10000.0, 10000.0, 10000.0, 100]
[100, 100, 100, 100, 100, 100, 100, 100]


In [111]:
model1 = gb.Model()

In [112]:
x = {}

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            x[i,j] = model1.addVar(vtype=gb.GRB.INTEGER,name = f'[x{station_name[i]},{station_name[j]}]')

In [113]:
h = {}


for i in range(len(MyOD)):

    h[i] = model1.addVar(vtype=gb.GRB.INTEGER,name = f'[h{station_name[i]}]')

In [114]:
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1 ))== quicksum(x[j,len(MyOD)-1]  for j in range(1,len(MyOD)-1,1) ))

<gurobi.Constr *Awaiting Model Update*>

aqws

In [115]:
for j in range(1,len(MyOD)-1,1):

    model1.addConstr(quicksum(x[i,j] for i in range(len(MyOD)) if i!= j)== quicksum(x[j,k] for k in range(len(MyOD) )if k!= j))

In [116]:
for i in range(1,len(MyOD)-1,1 ):
    model1.addConstr(quicksum(x[k,i] for k in range(1,len(MyOD),1) if k !=i) >= 1)

In [117]:
#main
Z1 = quicksum(int(MyOD[i][j])*x[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)
print(type(Z1))
model1.setObjective(Z1, sense=gb.GRB.MINIMIZE)

<class 'gurobipy.LinExpr'>


In [118]:
model1.update()

In [119]:
model1.display()

Minimize
100.0 [xd,V1] + 100.0 [xd,V2] + 100.0 [xd,V3] + 100.0 [xd,V4] + 100.0 [xd,V5]
+ 100.0 [xd,V6] + 100.0 [xd,a] + 100.0 [xV1,d] + 10000.0 [xV1,V2] + 10000.0 [xV1,V3]
+ [xV1,V4] + [xV1,V5] + 10000.0 [xV1,V6] + 100.0 [xV1,a] + 100.0 [xV2,d]
+ 10000.0 [xV2,V1] + 10000.0 [xV2,V3] + 10000.0 [xV2,V4] + 10000.0 [xV2,V5]
+ 10000.0 [xV2,V6] + 100.0 [xV2,a] + 100.0 [xV3,d] + 10000.0 [xV3,V1] + [xV3,V2]
+ 10000.0 [xV3,V4] + 10000.0 [xV3,V5] + 10000.0 [xV3,V6] + 100.0 [xV3,a] + 100.0 [xV4,d]
+ 10000.0 [xV4,V1] + 10000.0 [xV4,V2] + 10000.0 [xV4,V3] + 10000.0 [xV4,V5]
+ 10000.0 [xV4,V6] + 100.0 [xV4,a] + 100.0 [xV5,d] + 10000.0 [xV5,V1] + 10000.0 [xV5,V2]
+ 10000.0 [xV5,V3] + 10000.0 [xV5,V4] + 10000.0 [xV5,V6] + 100.0 [xV5,a] + 100.0 [xV6,d]
+ 10000.0 [xV6,V1] + [xV6,V2] + 10000.0 [xV6,V3] + 10000.0 [xV6,V4] + 10000.0 [xV6,V5]
+ 100.0 [xV6,a] + 100.0 [xa,d] + 100.0 [xa,V1] + 100.0 [xa,V2] + 100.0 [xa,V3]
+ 100.0 [xa,V4] + 100.0 [xa,V5] + 100.0 [xa,V6]
Subject To
R0: [xd,V1] + [xd,V2] + [xd,V3

C:\Users\zhuangz giant\AppData\Local\Temp\ipykernel_23968\2040429350.py:1: DeprecationWarning: Model.display() is deprecated
  model1.display()


In [120]:
print('\n')
model1.setParam(gb.GRB.Param.MIPGap, 0.005)
model1.setParam(gb.GRB.Param.TimeLimit, 3600*2)
model1.optimize()
# 可以输出求解的决策变量
try:
    if model1.ObjVal >= M*2:
        print(f'Model result of {model1.ObjVal:.2f}  exceed M:{M:.2f}')
        print('Result may out of node\\arc ability ')
    else:
        print(model1.ObjVal)
except Exception as e:
    print(e)
    print('\033[91m\nFatal! Model is INFEASIBLE.')

    if ISSme:
        model1.computeIIS()
        model1.write("model1.ilp")
    else:
        print('\033[5m\n ISS CHECK CANCEL \n\033[0m')



Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-14600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 13 rows, 64 columns and 132 nonzeros
Model fingerprint: 0x7eca729a
Variable types: 0 continuous, 64 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 20601.000000
Presolve removed 0 rows and 10 columns
Presolve time: 0.00s
Presolved: 13 rows, 54 columns, 132 nonzeros
Variable types: 0 continuous, 54 integer (0 binary)
Found heuristic solution: objective 10801.000000

Root relaxation: objective 8.020000e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj

In [121]:
#print(p[reflector[i], reflector[j],k].varName,p[reflector[i], reflector[j],k].x)

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            if x[i,j].x >= 0.001 :
                print(x[i,j].varName,x[i,j].x,x[i,j].x*MyOD[i][j])

[xV1,V4] 1.0 1.0
[xV2,d] 1.0 100.0
[xV3,d] 1.0 100.0
[xV4,d] 1.0 100.0
[xV5,d] 1.0 100.0
[xV6,V2] 1.0 1.0
[xa,V1] 1.0 100.0
[xa,V3] 1.0 100.0
[xa,V5] 1.0 100.0
[xa,V6] 1.0 100.0
